In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio

init_notebook_mode(connected=True)
plotly.__version__

'3.8.1'

In [2]:
tweets = pd.read_pickle('../datasets/TweetsDataset.pkl')
users = pd.read_pickle('../datasets/UsersDataset.pkl')
print(tweets.columns)
display(users.head(3))
tweets.head()

Index(['tweet_id', 'user_id', 'screen_name', 'complete_name', 'text',
       'hashtags', 'mencioned_user_id', 'mencioned_screen_name', 'is_reply',
       'reply_screen_name', 'reply_user_id', 'all_reply_screen_name',
       'all_reply_user_id', 'is_retweet', 'retweet_user_id',
       'retweet_screen_name', 'retweet_complete_name', 'retweet_count',
       'reply_count', 'favorite_count', 'num_hashtags', 'num_urls',
       'num_mentions', 'created_at', 'crawled_at'],
      dtype='object')


,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
index,,,,,,,,,,,,,,,
2300560809,2300560809,NMoraniB,Nicole Morani Brown,593,Gerencia General de @PuertoBsAs Ministerio de ...,"Buenos Aires, Argentina",False,False,0,2014-01-01 00:00:00,2019-06-29 23:04:21.059135,2377,1630,1510,False
1536923558,1536923558,FabianPereyra87,Fabian Pereyra,2532,,False,False,False,0,2013-06-01 00:00:00,2019-06-29 23:04:21.289518,4307,53,284,False
235491971,235491971,gsrcaballero,Gus Caballero,21635,"Papa de Cami y Rochi, FANA de La Academia, PRO...",False,False,False,0,2011-01-01 00:00:00,2019-06-29 23:04:21.030207,5683,1691,1360,False


,tweet_id,user_id,screen_name,complete_name,text,hashtags,mencioned_user_id,mencioned_screen_name,is_reply,reply_screen_name,...,retweet_screen_name,retweet_complete_name,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,crawled_at
1146580422798860288,1146580422798860288,291506013,lunaenpluton,Ocasi 💚,"“Fito Paaaaaeeeeezzzz, donde estaaaaaaaas?” Es...",[],[138072143],[FitoPaezMusica],False,NaN,...,NaN,NaN,0,1,17,0,0,1,2019-07-04 00:44:09,2019-07-08 11:53:25.184210
1145744849636876294,1145744849636876294,7764092,d3mian,Damian Allende / D3mian.com 👟 ✏️,-Le di de comer y dormí al bebe\n-Hice tarta g...,[],[],[],False,NaN,...,NaN,NaN,451,778,3481,0,0,0,2019-07-01 17:23:53,2019-07-11 19:13:55.772234
1148208276959584257,1148208276959584257,2312698567,cadenainforma,CADENA INFORMATIVA,Macri encabeza en Parque Norte el lanzamiento ...,[],[],[],False,NaN,...,NaN,NaN,5,1,9,0,2,0,2019-07-08 12:32:40,2019-07-08 11:53:27.285528
1145145462518898688,1145145462518898688,291506013,lunaenpluton,Ocasi 💚,“Quiero escuchar Fito Páez”. Tiene un año y oc...,[],[],[],False,NaN,...,NaN,NaN,0,1,13,0,0,0,2019-06-30 01:42:08,2019-07-08 11:53:41.788105
891083958712250368,891083958712250368,351400701,lzlptclss,luis islas,Cara de tener la conciencia rota,[],[],[],False,NaN,...,NaN,NaN,0,0,1,0,0,0,2017-07-28 23:52:25,2019-07-08 11:53:29.922502


In [3]:
#tweets_humans = tweets[tweets.Cat=='Humans']
#display(tweets_humans.columns)

#first = True
def getTimes(times):
    if(len(times) == 0):
        return np.array([0,0,0,0])
    return np.array([times.mean().seconds, times.median().seconds, times.std().seconds, times.quantile(0.25).seconds, times.quantile(0.75).seconds])

def getDist(var):
    if(len(var) == 0):
        return np.array([0,0,0,0])
    return np.array([var.mean(), var.median(), var.std(), var.quantile(0.25), var.quantile(0.75)])

def getWeekAv(tw):
    return len(tw) / len(tw.weekNum.unique())

def getDataTweets(twU):
    userId = twU.user_id.values[0]
    twU = twU.sort_values(by='created_at', ascending=0).fillna(0) #tweets del usuario
    dates = twU.created_at #aca lo que hago es guardar la columnoa como datetime

    twU.loc[:,'dayOfWeek'] = dates.dt.dayofweek # dia de la semana
    twU.loc[:,'hour'] = dates.dt.hour #hora del tweet
    twU.loc[:,'weekNum'] = dates.apply(lambda x: str(x.week) + '-' + str(x.year))

    tweetsUser = twU[twU.is_retweet==False] #tweets propios del usuario
    tweetsRT = twU[twU.is_retweet==True] #retweets

#     twU.loc[:,'interval'] = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp)
    twU.loc[:,'interval'] = pd.to_datetime(twU.created_at.shift(1)) - pd.to_datetime(twU.created_at)
    times = pd.to_datetime(twU.created_at.shift(1)) - pd.to_datetime(twU.created_at) # calculo distancia entre tweets

    tweetsUser.loc[:,'interval'] = pd.to_datetime(tweetsUser.created_at.shift(1)) - pd.to_datetime(tweetsUser.created_at)
    timesUser = pd.to_datetime(tweetsUser.created_at.shift(1)) - pd.to_datetime(tweetsUser.created_at) # calculo distancia entre tweets

    tweetsRT.loc[:,'interval'] = pd.to_datetime(tweetsRT.created_at.shift(1)) - pd.to_datetime(tweetsRT.created_at)
    timesRT = pd.to_datetime(tweetsRT.created_at.shift(1)) - pd.to_datetime(tweetsRT.created_at) # calculo distancia entre tweets
    
    mediaSemanas = np.around(twU.groupby('dayOfWeek').apply(getWeekAv), 6)
    
    #ver word_to_vect
    result = np.concatenate([
        [
            userId,
            twU.tweet_id.count(),
            len(twU.weekNum.unique())
        ],
        mediaSemanas,
        getTimes(times),
        getTimes(timesRT),
        getTimes(timesUser),
        getDist(tweetsUser.favorite_count.astype(int)) * 10000,
        getDist(tweetsUser.retweet_count.astype(int)) * 10000,
        getDist(tweetsUser.num_mentions.astype(int)) * 10000,
        getDist(tweetsUser.reply_count.astype(int)) * 10000,

    ])
    
    return result

res = tweets.groupby('user_id').apply(getDataTweets)
cols = np.concatenate(
    [
        
        [x+'_mean',x+'_median', x+'_std', x+'_fq', x+'_tq'] for x in
        ['times','timesRT','timesUser','favorite_count','retweet_count','num_mentions','reply_count']
    ]
)
cols = np.concatenate([
    ['user_id', 'tweetsCount', 'semanas', 'lun', 'mar', 'mie', 'jue', 'vie', 'sab', 'dom'],cols
])
res = pd.DataFrame(
    [x for x in res], columns = cols
)
res.index = res.user_id

print(len(res))

1531


In [5]:
res.to_pickle('../datasets/tweetsProcesados.pkl')